# Data Cleaning and database creation

### Upload to database file

In [11]:
import pandas as pd
import sqlite3

def insert_clean_csv_to_sqlite(
    csv_path: str,
    db_path: str,
    table_name: str,
    schema: str,
    replace_table: bool = False
):
    # Load the cleaned CSV
    df = pd.read_csv(csv_path)

    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Create table if needed (you define schema manually)
    cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({schema});")

    # Choose insertion mode
    if replace_table:
        insertion_mode = "replace"
    else:
        insertion_mode = "append"

    # Insert the data
    df.to_sql(table_name, conn, if_exists=insertion_mode, index=False)

    conn.commit()
    conn.close()
    print(f"✅ Inserted {len(df)} rows into table '{table_name}' in '{db_path}'.")



## Battles list

In [24]:
import pandas as pd
import re

def clean_battles_csv(
    csv_path: str,
    output_csv: str = "clean_battles.csv"
):
    # Load CSV
    df = pd.read_csv(csv_path)

    # Step 1: Clean battle_name
    def clean_battle_name(name):
        name = str(name)
        if "(" in name and ")" in name:
            match = re.search(r'\(([^)]+)\)', name)
            if match:
                content = match.group(1)
                if not content.isalpha():
                    name = re.sub(r'\([^)]*\)', '', name)
        return name.strip()

    df["battle_name"] = df["battle_name"].apply(clean_battle_name)

    # Step 2: Fill year column
    def extract_year(row):
        if pd.notnull(row["year"]):
            return row["year"]
        desc = str(row["description"])
        parts = desc.split('-')

        if len(parts) > 1 and any(x in parts[1] for x in ["BCE", "BC"]):
            try:
                year_candidate = int(re.findall(r'\d+', parts[1])[0])
                return -1 * year_candidate
            except:
                return None
        elif len(parts) > 0 and any(x in parts[0] for x in ["BCE", "BC"]):
            match = re.search(r'(\d+)\s*(BCE|BC)', parts[0])
            if match:
                return -1 * int(match.group(1))
        return None

    df["year"] = df.apply(extract_year, axis=1)

    # Step 3: Clean and fill conflict — only sanitize "war"
    def extract_conflict(row):
        conflict = row.get("conflict", None)
        if pd.notnull(conflict):
            raw = str(conflict)
        else:
            desc_parts = str(row["description"]).split('-')
            if len(desc_parts) > 2:
                raw = desc_parts[2]
            else:
                return None

        # Base cleanup
        raw = raw.strip()
        raw = re.sub(r'^[\d\?\s]*-\s*', '', raw)
        raw = re.sub(r'^[\d\s]*\)\-?', '', raw)
        raw = re.sub(r'^[\)\-,\s]+', '', raw)
        raw = re.sub(r'[\(\)]', '', raw)

        # Normalize "war"
        raw = re.sub(r'\bwars\b', 'war', raw, flags=re.IGNORECASE)
        raw = re.sub(r'\bwar war\b', 'war', raw, flags=re.IGNORECASE)
        raw = re.sub(r'\bwar\b', 'war', raw, flags=re.IGNORECASE)
        raw = re.sub(r'\bwar[.,\-_:;)]*\s*$', 'war', raw, flags=re.IGNORECASE)

        # Final whitespace cleanup
        raw = re.sub(r'\s+', ' ', raw).strip()

        return raw if raw else None

    df["conflict"] = df.apply(extract_conflict, axis=1)

    # Step 4: Format columns and handle missing values
    df["battle_id"] = pd.to_numeric(df["battle_id"], errors="coerce").astype("Int64")
    df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
    df["description"] = df["description"].replace("None", pd.NA).astype("string")
    df["conflict"] = df["conflict"].replace("None", pd.NA).astype("string")
    if "wiki_link" not in df.columns:
        df["wiki_link"] = pd.NA
    df["wiki_link"] = df["wiki_link"].replace("None", pd.NA).astype("string")

    # Save cleaned version to CSV
    df.to_csv(output_csv, index=False)
    print(f"✅ Cleaned CSV saved as '{output_csv}' with {len(df)} rows.")


In [25]:
clean_battles_csv('/Users/louis/Desktop/Coding/Github/WarCast/Data/Pre_clean/battles_list.csv')

✅ Cleaned CSV saved as 'clean_battles.csv' with 962 rows.


In [33]:
insert_clean_csv_to_sqlite(
    csv_path="/Users/louis/Desktop/Coding/Github/WarCast/Data/Clean/clean_battles.csv",
    db_path="/Users/louis/Desktop/Coding/Github/WarCast/Data/Database/warcast.db",
    table_name="battles",
    schema="""
        battle_id INTEGER PRIMARY KEY,
        battle_name TEXT,
        year INTEGER,
        description TEXT,
        conflict TEXT,
        wiki_link TEXT
    """,
    replace_table=True
)

✅ Inserted 962 rows into table 'battles' in '/Users/louis/Desktop/Coding/Github/WarCast/Data/Database/warcast.db'.


## Battle info

### Primary key resetting and column formatting

In [9]:
import pandas as pd
import re

def clean_participants_csv(
    csv_path: str = "battle_info.csv",
    output_path: str = "clean_battle_info.csv"
):
    """
    Clean the battle participants CSV file, with special focus on location data.
    
    Args:
        csv_path: Path to the input CSV file
        output_path: Path to save the cleaned CSV file
    """
    # Load CSV
    print(f"Loading data from {csv_path}...")
    df = pd.read_csv(csv_path)
    original_shape = df.shape
    print(f"Loaded {original_shape[0]} rows and {original_shape[1]} columns")
    
    # Step 1: Reset `participant_id` column
    if "participant_id" in df.columns:
        df.drop(columns=["participant_id"], inplace=True)
    df.insert(0, "participant_id", df.index + 1)
    
    # Step 2: Clean city and country location columns
    print("\nCleaning location data...")
    
    # Before counts
    city_na_before = df['city_battle_location'].isna().sum()
    country_na_before = df['country_battle_location'].isna().sum()
    print(f"Missing values before cleaning - Cities: {city_na_before}, Countries: {country_na_before}")
    
    # Pre-clean coordinates from raw data
    df['city_battle_location'] = df['city_battle_location'].apply(remove_coordinate_patterns)
    df['country_battle_location'] = df['country_battle_location'].apply(remove_coordinate_patterns)
    
    # Clean locations and extract city/country when possible
    location_pairs = []
    for i, row in df.iterrows():
        city, country = extract_city_country(row['city_battle_location'], row['country_battle_location'])
        location_pairs.append((city, country))
    
    # Apply the extracted city and country values
    df['city_battle_location'] = [pair[0] for pair in location_pairs]
    df['country_battle_location'] = [pair[1] for pair in location_pairs]
    
    # Final sanitization of city and country names
    df['city_battle_location'] = df['city_battle_location'].apply(sanitize_name)
    df['country_battle_location'] = df['country_battle_location'].apply(sanitize_name)
    
    # Extra pass to clean up any remaining coordinate patterns
    df['city_battle_location'] = df['city_battle_location'].apply(remove_coordinate_patterns)
    df['country_battle_location'] = df['country_battle_location'].apply(remove_coordinate_patterns)
    
    # Remove unwanted geographic terms like "Littoral"
    df['city_battle_location'] = df['city_battle_location'].apply(remove_geographic_terms)
    df['country_battle_location'] = df['country_battle_location'].apply(remove_geographic_terms)
    
    # Final standardization of country names
    df['country_battle_location'] = df['country_battle_location'].apply(standardize_country_name)
    
    # After counts
    city_na_after = df['city_battle_location'].isna().sum()
    country_na_after = df['country_battle_location'].isna().sum()
    print(f"Missing values after cleaning - Cities: {city_na_after}, Countries: {country_na_after}")
    
    # Step 3: Enforce specific formats for other columns
    print("\nEnforcing data types...")
    df["participant_id"] = pd.to_numeric(df["participant_id"], downcast="integer")
    df["battle_id"] = pd.to_numeric(df["battle_id"], errors="coerce").astype("Int64")
    df["country"] = df["country"].astype("string")
    df["troops"] = pd.to_numeric(df["troops"], errors="coerce").astype("Int64")
    df["deaths"] = pd.to_numeric(df["deaths"], errors="coerce").astype("Int64")
    df["result"] = df["result"].astype("string")
    
    # Convert location columns to string type
    df["city_battle_location"] = df["city_battle_location"].astype("string")
    df["country_battle_location"] = df["country_battle_location"].astype("string")
    
    # Step 4: Save cleaned file
    df.to_csv(output_path, index=False)
    print(f"\n✅ Cleaned participant data saved to '{output_path}' with {len(df)} rows.")
    
    # Print some statistics
    print(f"\nLocation data statistics:")
    print(f"- Rows with valid city: {df['city_battle_location'].notna().sum()} ({df['city_battle_location'].notna().sum()/len(df)*100:.1f}%)")
    print(f"- Rows with valid country: {df['country_battle_location'].notna().sum()} ({df['country_battle_location'].notna().sum()/len(df)*100:.1f}%)")
    print(f"- Rows with both city and country: {(df['city_battle_location'].notna() & df['country_battle_location'].notna()).sum()} ({(df['city_battle_location'].notna() & df['country_battle_location'].notna()).sum()/len(df)*100:.1f}%)")
    
    # Return the cleaned dataframe
    return df

def extract_city_country(city_val, country_val):
    """
    Extract city and country from location values, handling complex cases.
    
    For example, with "Garibpur, Dhaka, Bangladesh", it will extract
    "Dhaka" as the city (second-to-last part) and "Bangladesh" as the country (last part).
    """
    # Common countries list for validation
    common_countries = [
        "Afghanistan", "Albania", "Algeria", "Argentina", "Australia", "Austria", "Bangladesh",
        "Belgium", "Brazil", "Bulgaria", "Canada", "China", "Croatia", "Czech Republic", 
        "Denmark", "Egypt", "Finland", "France", "Germany", "Greece", "Hungary", "India", 
        "Indonesia", "Iran", "Iraq", "Ireland", "Israel", "Italy", "Japan", "Korea", 
        "Malaysia", "Mexico", "Morocco", "Netherlands", "New Zealand", "Norway", "Pakistan", 
        "Philippines", "Poland", "Portugal", "Romania", "Russia", "Saudi Arabia", "Serbia", 
        "Singapore", "South Africa", "Spain", "Sweden", "Switzerland", "Syria", "Thailand", 
        "Turkey", "Ukraine", "United Arab Emirates", "United Kingdom", "Great Britain", "England",
        "United States", "America", "Vietnam", "Yemen", "Zimbabwe", "Soviet Union", 
        "Ottoman Empire", "Prussia", "Holy Roman Empire", "Byzantine Empire", "Roman Empire",
        # Add countries with directional components
        "South Sudan", "East Timor", "North Macedonia", "North Korea", "South Korea", 
        "Western Sahara", "Eastern Republic of Uruguay", "United Kingdom of Great Britain and Northern Ireland"
    ]
    
    # US States list for standardization
    us_states = [
        "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", 
        "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", 
        "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", 
        "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", 
        "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", 
        "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", 
        "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", 
        "Wisconsin", "Wyoming"
    ]
    
    # Initialize results with the original values (possibly NaN)
    city_result = city_val
    country_result = country_val
    
    # First clean the values
    city_cleaned = clean_location_text(city_result)
    country_cleaned = clean_location_text(country_result)
    
    # If a US state is in the country column, replace with "United States"
    if pd.notna(country_cleaned) and country_cleaned in us_states:
        country_cleaned = "United States"
    
    # If one is None but the other has a comma, try to extract both from the one value
    if pd.notna(city_cleaned) and ',' in city_cleaned and pd.isna(country_cleaned):
        # City has multiple parts, try to extract both from it
        parts = [p.strip() for p in city_cleaned.split(',')]
        
        # 1. Try to identify a country in the parts
        country_from_parts = None
        for part in reversed(parts):  # Check from the end
            if part in common_countries:
                country_from_parts = part
                break
        
        if country_from_parts:
            country_result = country_from_parts
            # Get the index of the country
            country_index = next((i for i, part in enumerate(parts) if part == country_from_parts), -1)
            
            # If it's the last part and there's at least one part before it
            if country_index == len(parts) - 1 and country_index > 0:
                # Use the second-to-last part as the city
                city_result = parts[country_index - 1]
            else:
                # Otherwise use the first part as the city
                city_result = parts[0]
        else:
            # No country found, assume last part is country and part before is city
            if len(parts) >= 2:
                country_result = parts[-1]
                city_result = parts[-2]
            else:
                city_result = parts[0]
    
    elif pd.notna(country_cleaned) and ',' in country_cleaned and pd.isna(city_cleaned):
        # Country has multiple parts, try to extract both from it
        parts = [p.strip() for p in country_cleaned.split(',')]
        
        # Assume last part is country and part before is city
        if len(parts) >= 2:
            country_result = parts[-1]
            city_result = parts[-2]
        else:
            country_result = parts[0]
    
    # Final cleaning of the individual values
    city_result = clean_city_name(city_result)
    country_result = clean_country_name(country_result)
    
    # If city is a country and country is None, swap them
    if pd.notna(city_result) and pd.isna(country_result) and city_result in common_countries:
        country_result = city_result
        city_result = None
    
    # If both have values but city is a country and country isn't, swap them
    if pd.notna(city_result) and pd.notna(country_result):
        if city_result in common_countries and country_result not in common_countries:
            temp = city_result
            city_result = country_result
            country_result = temp
    
    # If country is a US state, convert to United States
    if pd.notna(country_result) and country_result in us_states:
        country_result = "United States"
    
    # Additional cleaning to handle remaining coordinate patterns
    city_result = remove_coordinate_patterns(city_result)
    country_result = remove_coordinate_patterns(country_result)
    
    return city_result, country_result

def remove_coordinate_patterns(text):
    """
    Very aggressively remove any coordinate-like patterns from text.
    """
    if pd.isna(text):
        return None
        
    # Convert to string
    text = str(text)
    
    # Check specifically for the "N E" pattern at the end (without periods)
    simple_end_pattern = r'^(.*?)(?:\s+[NESW]\s+[NESW](?:\s|$)).*$'
    simple_match = re.search(simple_end_pattern, text)
    if simple_match and simple_match.group(1).strip():
        return simple_match.group(1).strip()
    
    # Check for coordinate-like patterns
    coordinate_markers = ['.N', '.E', '.S', '.W', ' N ', ' E ', ' S ', ' W ']
    if any(marker in text for marker in coordinate_markers) or text.rstrip().endswith((' N', ' E', ' S', ' W')):
        # Check for repeated patterns like '.N .W' or '.N .E'
        patterns = [
            r'\.[NESW]\s+\.[NESW]',  # Pattern like '.N .W'
            r'\s[NESW]\s+\.[NESW]',  # Pattern like 'N .W'
            r'\.[NESW]\s+[NESW]\s',  # Pattern like '.N W'
            r'\s[NESW]\s+[NESW]\s',  # Pattern like 'N W'
            r'\s[NESW]\s+[NESW]$',   # Pattern like 'N E' at the end
        ]
        
        for pattern in patterns:
            if re.search(pattern, text):
                # Find where the pattern starts
                match = re.search(pattern, text)
                if match:
                    # Return only the text before the pattern
                    return text[:match.start()].strip()
        
        # Look for all compass directions with periods or spaces around them
        pattern_all_dirs = r'(?<!\w)[\.\s][NESW][\.\s-]'
        if re.search(pattern_all_dirs, text):
            # Find the first occurrence
            match = re.search(pattern_all_dirs, text)
            if match:
                # Return only the text before the first occurrence
                return text[:match.start()].strip()
                
        # Check for single compass direction at the end
        end_dir_pattern = r'^(.*?)\s+[NESW]$'
        end_match = re.search(end_dir_pattern, text)
        if end_match:
            return end_match.group(1).strip()
    
    # Look for any standalone compass directions at the end of the text
    # This catches simpler cases like "Thessaly .N"
    for dir_letter in ['N', 'E', 'S', 'W']:
        pattern = rf'(.*?)\s+\.{dir_letter}.*'
        match = re.search(pattern, text)
        if match:
            return match.group(1).strip()
    
    # Remove any trailing dash which might be part of coordinate notation
    text = re.sub(r'\s+-$', '', text)
    
    # Remove single periods, which might be part of coordinate notation
    text = re.sub(r'\s+\.\s+', ' ', text)
    
    return text

def clean_location_text(loc_val):
    """Initial cleaning of location text."""
    if pd.isna(loc_val):
        return None
    
    # Convert to string
    loc_val = str(loc_val)
    
    # Remove coordinate patterns first
    loc_val = remove_coordinate_patterns(loc_val)
    if pd.isna(loc_val):
        return None
    
    # Special case: If the text contains coordinates or special symbols,
    # extract just the place name at the beginning
    # We'll use a more aggressive approach to identify place names
    if any(symbol in loc_val for symbol in ['°', '¬', '‚', 'Ä', 'Ô', 'ª', 'ø', '.N', '.E', '.S', '.W']):
        # Find the first occurrence of degree symbol, coordinates or special characters
        coord_indicators = ['°', '¬', '‚', 'Ä', 'Ô', 'ª', 'ø']
        
        # First try to find the index where coordinates start
        coord_pos = len(loc_val)  # Default to end of string
        for indicator in coord_indicators:
            if indicator in loc_val:
                pos = loc_val.find(indicator)
                if pos < coord_pos:
                    coord_pos = pos
        
        # Look for patterns like ".N" or ".E"
        direction_patterns = ['.N', '.E', '.S', '.W']
        for pattern in direction_patterns:
            if pattern in loc_val:
                pos = loc_val.find(pattern)
                if pos < coord_pos:
                    coord_pos = pos
        
        # If we found any coordinate indicators, extract the part before them
        if coord_pos < len(loc_val):
            cleaned = loc_val[:coord_pos].strip()
            # If we have something, return it
            if cleaned:
                return cleaned
    
    # Continue with regular cleaning if no coordinates were found
    # or if the extraction resulted in an empty string
    
    # Remove text in brackets, parentheses
    cleaned = re.sub(r'\([^)]*\)', '', loc_val)
    cleaned = re.sub(r'\[[^\]]*\]', '', cleaned)
    
    # Remove footnote references
    cleaned = re.sub(r'\[\d+\]', '', cleaned)
    
    # Remove coordinate patterns more aggressively
    # This will catch both the special symbols and the direction letters
    coordinate_pattern = r'[¬°‚Ä≤‚Ä≥ÔªøNESW\d\.\-\+\'\"°′″]+|(?<=\s)[NESW](?=[\s\.,;])|(?<=\s)[NESW]$'
    cleaned = re.sub(coordinate_pattern, ' ', cleaned)
    
    # Remove various coordinate separators: slashes, periods, commas between numbers
    cleaned = re.sub(r'[\\/]', ' ', cleaned)
    
    # Remove phrases like "near", "present-day", etc.
    cleaned = re.sub(r'\bnear\b|\bpresent-day\b|\bmodern-day\b|\bmodern\b|\bvicinity of\b|\boff\b|\barea\b', '', cleaned, flags=re.IGNORECASE)
    
    # Remove any text after irrelevant markers
    irrelevant_markers = ['while more', 'identified as', 'km west', 'km east', 'km north', 'km south']
    for marker in irrelevant_markers:
        if marker in cleaned:
            pos = cleaned.find(marker)
            cleaned = cleaned[:pos].strip()
    
    # Remove common prefixes/suffixes
    cleaned = re.sub(r'^the\s+', '', cleaned, flags=re.IGNORECASE)
    
    # Remove excess whitespace
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    
    # Replace multiple periods or dots
    cleaned = re.sub(r'\.{2,}', '', cleaned)
    
    # Final check: If the cleaned string only has punctuation or is very short, return None
    if not re.search(r'[a-zA-Z]', cleaned) or len(cleaned) <= 1:
        return None
    
    return cleaned

def clean_city_name(city):
    """Clean city name."""
    if pd.isna(city):
        return None
    
    # Convert to string
    city = str(city)
    
    # Remove coordinate patterns first
    city = remove_coordinate_patterns(city)
    if pd.isna(city):
        return None
    
    # Aggressively remove coordinate and special characters
    city = re.sub(r'[¬°‚Ä≤‚Ä≥Ôªø]', '', city)
    
    # Remove standalone direction indicators (N,E,S,W)
    city = re.sub(r'\s+[NESW]\s+|\s+[NESW]$|^[NESW]\s+', ' ', city)
    
    # Remove digits if not all digits
    if not city.isdigit() and any(c.isdigit() for c in city):
        city = ''.join(c for c in city if not c.isdigit())
        city = re.sub(r'\s+', ' ', city).strip()
    
    # Set to None if it's only digits
    if city.isdigit():
        return None
    
    # Clean up directional prefixes (for cities, not countries)
    city = re.sub(r'^(northern|southern|eastern|western|north|south|east|west|upper|lower|southeastern|southwestern|northeastern|northwestern)\s+', '', city, flags=re.IGNORECASE)
    
    # Remove multiple periods or dots
    city = re.sub(r'\.{2,}', '', city)
    
    # Set to None if too short after cleaning
    if len(city) <= 1:
        return None
        
    return city

def clean_country_name(country):
    """Clean country name."""
    if pd.isna(country):
        return None
    
    # Convert to string
    country = str(country)
    
    # Remove coordinate patterns first
    country = remove_coordinate_patterns(country)
    if pd.isna(country):
        return None
    
    # Aggressively remove coordinate and special characters
    country = re.sub(r'[¬°‚Ä≤‚Ä≥Ôªø]', '', country)
    
    # Remove standalone direction indicators (N,E,S,W)
    country = re.sub(r'\s+[NESW]\s+|\s+[NESW]$|^[NESW]\s+', ' ', country)
    
    # List of valid country names with directional components
    directional_countries = [
        "South Africa", "South Sudan", "East Timor", "North Macedonia", 
        "North Korea", "South Korea", "Western Sahara",
        "Eastern Republic of Uruguay", "United Kingdom of Great Britain and Northern Ireland"
    ]
    
    # Check if the country is already a recognized country with direction
    for valid_country in directional_countries:
        if valid_country.lower() in country.lower():
            # Return the standardized form
            return valid_country
    
    # US States list for standardization
    us_states = [
        "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", 
        "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", 
        "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", 
        "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", 
        "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", 
        "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", 
        "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", 
        "Wisconsin", "Wyoming"
    ]
    
    # Replace US States with "United States"
    if country in us_states:
        return "United States"
    
    # Remove digits if not all digits
    if not country.isdigit() and any(c.isdigit() for c in country):
        country = ''.join(c for c in country if not c.isdigit())
        country = re.sub(r'\s+', ' ', country).strip()
    
    # Set to None if it's only digits
    if country.isdigit():
        return None
    
    # Clean up directional prefixes ONLY for non-recognized countries
    if not any(valid_country.lower() in country.lower() for valid_country in directional_countries):
        country = re.sub(r'^(northern|southern|eastern|western|north|south|east|west|upper|lower|southeastern|southwestern|northeastern|northwestern)\s+', '', country, flags=re.IGNORECASE)
    
    # Remove multiple periods or dots
    country = re.sub(r'\.{2,}', '', country)
    
    # Set to None if too short after cleaning
    if len(country) <= 1:
        return None
    
    # Standardize country names
    country = standardize_country_name(country)
        
    return country

def standardize_country_name(country):
    """Standardize country names to ensure consistency."""
    if pd.isna(country):
        return None
    
    # Convert to string
    country = str(country)
    
    # Replace common variations/abbreviations
    country_replacements = {
        "US": "United States",
        "U.S.": "United States",
        "U.S.A.": "United States",
        "USA": "United States",
        "America": "United States",
        "UK": "United Kingdom",
        "U.K.": "United Kingdom",
        "Britain": "United Kingdom",
        "Great Britain": "United Kingdom",
        "England": "United Kingdom", 
        "Soviet Union": "Russia",
        "USSR": "Russia",
        "U.S.S.R.": "Russia"
    }
    
    # Check for exact matches
    if country in country_replacements:
        return country_replacements[country]
    
    # Check for case-insensitive matches
    for key, value in country_replacements.items():
        if country.lower() == key.lower():
            return value
    
    return country

def remove_geographic_terms(name):
    """Remove unwanted geographic terms from location names."""
    if pd.isna(name):
        return None
    
    # Convert to string
    name = str(name)
    
    # List of geographic terms to remove
    geographic_terms = [
        r'\blittoral\b',
        r'\bplain\b',
        r'\bplateau\b',
        r'\bbasin\b',
        r'\bdelta\b',
        r'\bestuary\b',
        r'\bpeninsula\b',
        r'\bisthmus\b',
        r'\bstrait\b'
    ]
    
    # Remove each term (case insensitive)
    for term in geographic_terms:
        name = re.sub(term, '', name, flags=re.IGNORECASE)
    
    # Clean up any resulting extra spaces
    name = re.sub(r'\s+', ' ', name).strip()
    name = re.sub(r',\s*$', '', name)  # Remove trailing comma
    name = re.sub(r'^\s*,', '', name)  # Remove leading comma
    
    # Convert to None if empty
    if not name:
        return None
    
    return name

def sanitize_name(name):
    """Final sanitization of names to ensure they are clean and standardized."""
    if pd.isna(name):
        return None
    
    # Convert to string
    name = str(name)
    
    # Look for and remove patterns of standalone compass directions
    name = remove_coordinate_patterns(name)
    if pd.isna(name):
        return None
    
    # Trim whitespace
    name = name.strip()
    
    # Remove any standalone compass directions (N, E, S, W)
    name = re.sub(r'\s+[NESW](?=\s|$)', '', name)
    name = re.sub(r'\s+[NESW]\s+', ' ', name)
    name = re.sub(r'\s+[NESW]\.$', '', name)
    name = re.sub(r'\.+[NESW]', '', name)
    
    # Remove any remaining special characters
    name = re.sub(r'[^\w\s,.-]', '', name)
    
    # Remove duplicate spaces
    name = re.sub(r'\s+', ' ', name).strip()
    
    # Remove extra periods and trailing punctuation
    name = re.sub(r'\.{2,}', '.', name)
    name = re.sub(r'[.,;:]+$', '', name).strip()
    
    # Convert to None if empty
    if not name:
        return None
    
    return name

# Run the cleaning script if executed directly
if __name__ == "__main__":
    clean_participants_csv("battle_info.csv", "clean_battle_info.csv")

Loading data from battle_info.csv...
Loaded 3184 rows and 8 columns

Cleaning location data...
Missing values before cleaning - Cities: 0, Countries: 383
Missing values after cleaning - Cities: 2, Countries: 383

Enforcing data types...

✅ Cleaned participant data saved to 'clean_battle_info.csv' with 3184 rows.

Location data statistics:
- Rows with valid city: 3182 (99.9%)
- Rows with valid country: 2801 (88.0%)
- Rows with both city and country: 2799 (87.9%)


### Upload to the database

In [34]:
insert_clean_csv_to_sqlite(
    csv_path='/Users/louis/Desktop/Coding/Github/WarCast/Data/Clean/clean_battle_info.csv',
    db_path='/Users/louis/Desktop/Coding/Github/WarCast/Data/Database/warcast.db',
    table_name="battle_info",
    schema="""
        participant_id INTEGER PRIMARY KEY,
        battle_id INTEGER,
        country TEXT,
        troops INTEGER,
        deaths INTEGER,
        result TEXT,
        city_battle_location TEXT,
        country_battle_location TEXT
    """,
    replace_table=True
)

✅ Inserted 3184 rows into table 'battle_info' in '/Users/louis/Desktop/Coding/Github/WarCast/Data/Database/warcast.db'.


## GDP

In [39]:
import pandas as pd
import sqlite3

def process_gdp_excel_to_sqlite(
    excel_path: str,
    db_path: str = '/Users/louis/Desktop/Coding/Github/WarCast/Data/Database/warcast.db',
    table_name: str = "gdp"
):
    # Step 1: Load Excel
    df = pd.read_excel(excel_path)
    
    # Step 2: Melt into long format
    df_melted = df.melt(
        id_vars=["Country Name"],
        var_name="year",
        value_name="gdp"
    )
    
    # Step 3: Clean
    df_melted.rename(columns={"Country Name": "country"}, inplace=True)
    df_melted = df_melted.dropna(subset=["gdp"])
    df_melted["year"] = pd.to_numeric(df_melted["year"], errors="coerce").dropna().astype(int)
    df_melted["gdp"] = pd.to_numeric(df_melted["gdp"], errors="coerce").round(2)

    # Step 4: Add surrogate key (optional if you're going to use AUTOINCREMENT in SQL)
    df_melted = df_melted.sort_values(by=["country", "year"]).reset_index(drop=True)
    df_melted.insert(0, "id", df_melted.index + 1)

    # Step 5: Save cleaned CSV (optional but useful for inspection)
    df_melted.to_csv("clean_gdp.csv", index=False)

    # Step 6: Create or insert into SQLite
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Create table (safe to run repeatedly due to IF NOT EXISTS)
    cursor.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY,
        country TEXT,
        year INTEGER,
        gdp REAL,
        UNIQUE (country, year)
    );
    """)

    # Insert into database
    df_melted.to_sql(table_name, conn, if_exists="append", index=False)

    conn.commit()
    conn.close()

    print(f"✅ Processed and saved GDP data to table '{table_name}' in {db_path}")


In [41]:
process_gdp_excel_to_sqlite('/Users/louis/Desktop/Coding/Github/WarCast/Data/Pre_clean/GDP per country_1988-2022.xlsx')

✅ Processed and saved GDP data to table 'gdp' in /Users/louis/Desktop/Coding/Github/WarCast/Data/Database/warcast.db


## Political regime

In [42]:
import pandas as pd
import sqlite3

# Mapping from code to regime label
REGIME_MAP = {
    0: "closed autocracy",
    1: "electoral autocracy",
    2: "electoral democracy",
    3: "liberal democracy"
}

def process_political_regime_to_sqlite(
    csv_path: str = '',
    db_path: str = "/Users/louis/Desktop/Coding/Github/WarCast/Data/Database/warcast.db",
    table_name: str = "political_regime"
):
    # Step 1: Load CSV
    df = pd.read_csv(csv_path)

    # Step 2: Rename relevant columns
    df = df.rename(columns={
        "Entity": "country",
        "Year": "year",
        df.columns[-1]: "regime_code"  # Last column is assumed to be the regime score
    })

    # Step 3: Map regime code to readable labels
    df["regime_type"] = df["regime_code"].map(REGIME_MAP)

    # Step 4: Sort and add surrogate ID
    df = df.sort_values(by=["country", "year"]).reset_index(drop=True)
    df.insert(0, "id", df.index + 1)

    # Step 5: Save full cleaned CSV including 'Code'
    df.to_csv("clean_political_regime.csv", index=False)

    # Step 6: Connect to SQLite and create table
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY,
        country TEXT,
        year INTEGER,
        regime_code INTEGER,
        regime_type TEXT,
        UNIQUE (country, year)
    );
    """)

    # Step 7: Insert only relevant columns into SQL
    df_sql = df[["id", "country", "year", "regime_code", "regime_type"]]
    try:
        df_sql.to_sql(table_name, conn, if_exists="append", index=False)
        print(f"✅ Inserted data into table '{table_name}' in {db_path}")
    except Exception as e:
        print(f"❌ Failed to insert data: {e}")

    conn.commit()
    conn.close()


In [43]:
process_political_regime_to_sqlite('/Users/louis/Desktop/Coding/Github/WarCast/Data/Raw/political-regime/political-regime.csv')

✅ Inserted data into table 'political_regime' in /Users/louis/Desktop/Coding/Github/WarCast/Data/Database/warcast.db


## Population

In [45]:
import pandas as pd

def clean_population_csv(
    csv_path: str = "",
    output_path: str = "clean_population.csv"
):
    # Load the raw file
    df = pd.read_csv(csv_path)

    # Step 1: Rename columns
    df.rename(columns={
        "Entity": "Country",
        "Population (historical)": "Population"
    }, inplace=True)

    # Step 2: Enforce data types
    df["Country"] = df["Country"].astype("string")
    df["Code"] = df["Code"].astype("string")
    df["Year"] = pd.to_numeric(df["Year"], errors="coerce").astype("Int64")
    df["Population"] = pd.to_numeric(df["Population"], errors="coerce").astype("Int64")

    # Step 3: Save cleaned file
    df.to_csv(output_path, index=False)
    print(f"✅ Cleaned population data saved to '{output_path}' with {len(df)} rows.")


In [46]:
clean_population_csv('/Users/louis/Desktop/Coding/Github/WarCast/Raw Data/population/population.csv')

✅ Cleaned population data saved to 'clean_population.csv' with 59177 rows.


### Upload to database

In [45]:
insert_clean_csv_to_sqlite(
    csv_path='/Users/louis/Desktop/Coding/Github/WarCast/Data/Clean/clean_population.csv',
    db_path="/Users/louis/Desktop/Coding/Github/WarCast/Data/Database/warcast.db",
    table_name="population",
    schema="""
        id INTEGER PRIMARY KEY,
        Country TEXT,
        Code TEXT,
        Year INTEGER,
        Population INTEGER
    """,
    replace_table=True
)

✅ Inserted 59177 rows into table 'population' in '/Users/louis/Desktop/Coding/Github/WarCast/Data/Database/warcast.db'.


## Corruption Perception Index

In [55]:
import pandas as pd

def clean_cpi_scores(
    excel_path: str,
    output_path: str = "clean_cpi.csv"
):
    # Load Excel
    df = pd.read_csv(excel_path,encoding='latin1')

    # Step 1: Keep only country info and CPI score columns
    cpi_cols = [col for col in df.columns if "CPI score" in col]
    base_cols = ["Country", "ISO3"]
    df = df[base_cols + cpi_cols]

    # Step 2: Melt to long format
    df_long = df.melt(
        id_vars=["Country", "ISO3"],
        var_name="year",
        value_name="cpi_score"
    )

    # Step 3: Clean year column from 'CPI score YYYY' → YYYY
    df_long["year"] = df_long["year"].str.extract(r'(\d{4})').astype("Int64")

    # Step 4: Format columns
    df_long = df_long.rename(columns={"Country": "country", "ISO3": "code"})
    df_long["country"] = df_long["country"].astype("string")
    df_long["code"] = df_long["code"].astype("string")
    df_long["cpi_score"] = pd.to_numeric(df_long["cpi_score"], errors="coerce").astype("Int64")

    # Step 5: Add auto ID
    df_long = df_long.sort_values(by=["country", "year"]).reset_index(drop=True)
    df_long.insert(0, "id", df_long.index + 1)

    # Step 6: Save cleaned file
    df_long.to_csv(output_path, index=False)
    print(f"✅ Cleaned CPI data saved to '{output_path}' with {len(df_long)} rows.")

In [57]:
clean_cpi_scores('/Users/louis/Desktop/Coding/Github/WarCast/Pre-clean data/CPI2020_GlobalTablesTS_210125.csv')

✅ Cleaned CPI data saved to 'clean_cpi.csv' with 1260 rows.


### Upload to database

In [46]:
insert_clean_csv_to_sqlite(
    csv_path='/Users/louis/Desktop/Coding/Github/WarCast/Data/Clean/clean_cpi.csv',
    db_path="/Users/louis/Desktop/Coding/Github/WarCast/Data/Database/warcast.db",
    table_name="cpi",
    schema="""
        id INTEGER PRIMARY KEY,
        country TEXT,
        code TEXT,
        year INTEGER,
        cpi_score INTEGER
    """,
    replace_table=True
)

✅ Inserted 1260 rows into table 'cpi' in '/Users/louis/Desktop/Coding/Github/WarCast/Data/Database/warcast.db'.


## Military investment

In [28]:
import pandas as pd

def clean_military_percent_gdp(
    csv_path: str = "Military percent GDP.csv",
    output_path: str = "clean_military_gdp.csv"
):
    # Load raw CSV
    df = pd.read_csv(csv_path)

    # Step 1: Drop unused columns
    df = df.drop(columns=["Indicator Name", "Indicator Code"], errors="ignore")

    # Step 2: Melt into tall format
    df_long = df.melt(
        id_vars=["Country Name", "Country Code"],
        var_name="year",
        value_name="military_percent_gdp"
    )

    # Step 3: Rename columns and enforce types
    df_long = df_long.rename(columns={
        "Country Name": "country",
        "Country Code": "code"
    })

    df_long["country"] = df_long["country"].astype("string")
    df_long["code"] = df_long["code"].astype("string")
    df_long["year"] = pd.to_numeric(df_long["year"], errors="coerce").astype("Int64")
    df_long["military_percent_gdp"] = pd.to_numeric(df_long["military_percent_gdp"], errors="coerce").astype("float")

    # Step 4: Add auto ID
    df_long = df_long.sort_values(by=["country", "year"]).reset_index(drop=True)
    df_long.insert(0, "id", df_long.index + 1)

    # Step 5: Save cleaned version
    df_long.to_csv(output_path, index=False)
    print(f"✅ Cleaned military GDP data saved to '{output_path}' with {len(df_long)} rows.")


In [29]:
clean_military_percent_gdp('/Users/louis/Desktop/Coding/Github/WarCast/Data/Pre_clean/Military percent GDP.csv')

✅ Cleaned military GDP data saved to 'clean_military_gdp.csv' with 17024 rows.


### Upload to the database

In [47]:
insert_clean_csv_to_sqlite(
    csv_path='/Users/louis/Desktop/Coding/Github/WarCast/Data/Clean/clean_military_gdp.csv',
    db_path="/Users/louis/Desktop/Coding/Github/WarCast/Data/Database/warcast.db",
    table_name="gdp_military",
    schema="""
        id INTEGER PRIMARY KEY,
        country TEXT,
        code TEXT,
        year INTEGER,
        military_percent_gdp REAL
    """,
    replace_table=True
)

✅ Inserted 17024 rows into table 'gdp_military' in '/Users/louis/Desktop/Coding/Github/WarCast/Data/Database/warcast.db'.
